# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120058e+02     1.588629e+00
 * time: 0.6950821876525879
     1     1.090217e+01     8.532932e-01
 * time: 1.8696889877319336
     2    -1.178755e+01     1.043034e+00
 * time: 2.422194004058838
     3    -3.402328e+01     7.510281e-01
 * time: 3.25360107421875
     4    -4.762343e+01     5.458149e-01
 * time: 4.088795185089111
     5    -5.712128e+01     2.346842e-01
 * time: 4.9198009967803955
     6    -5.991681e+01     1.343063e-01
 * time: 5.458863973617554
     7    -6.097842e+01     6.388954e-02
 * time: 6.044905185699463
     8    -6.139923e+01     4.999352e-02
 * time: 6.592895030975342
     9    -6.165323e+01     5.302766e-02
 * time: 7.140709161758423
    10    -6.185036e+01     2.846209e-02
 * time: 7.676805019378662
    11    -6.199597e+01     2.305292e-02
 * time: 8.209609985351562
    12    -6.207638e+01     1.672773e-02
 * time: 8.751085996627808
    13    -6.213169e+01     1.501447e-02
 * time: 9.32360315322876
    14

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671060
    AtomicLocal         -18.8557648
    AtomicNonlocal      14.8522631
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485359 
    Xc                  -19.3336815

    total               -62.261666460066